In [14]:
import os
os.environ['USE_PYGEOS'] = '0'
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols 
from rasterstats import zonal_stats
import rasterio
import contextily as cx
from rasterio.plot import show
#import osmnx as ox
from affine import Affine
import rioxarray as rx
from multiprocessing import Pool, cpu_count
from shapely.ops import unary_union
import numpy as np
from osgeo import gdal
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
from pathlib import Path

import sys
sys.path.append("..")

from src.data.make_dataset import *

In [3]:

#read in municipalities, select the one of interest
#municipalities
munis_fp = "K:\\DataServices\\Datasets\\Boundaries\\Spatial\\mapc_towns_poly.shp"
munis = gpd.read_file(munis_fp)

lclu_gdb = "K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Project\\MS4_Tool_Preprocessing.gdb"

ms4_gdb = 'K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Project\\MS4_Tool_Preprocessing.gdb'
mapc_boundary = gpd.read_file(ms4_gdb, layer='MAPCBoundary')



In [8]:
#prep parcels
#get a public parcels layer from 3a

public_fp = 'K:\\DataServices\\Projects\\Current_Projects\\Housing\\Section_3A\\Compliance_Projects\\_State_Data\\_Reference_Layers\\Excluded_Land_Public_and_Institutional_Land.shp'
public_inst_excluded_land = gpd.read_file(public_fp)



,MAP_PAR_ID,LOC_ID,POLY_TYPE,MAP_NO,SOURCE,PLAN_ID,LAST_EDIT,BND_CHK,NO_MATCH,TOWN_ID,...,SHAPE_Leng,SHAPE_Area,USE_CODE,USE_DESC,GROUP_,FULL_STR,CITY,OWNER1,YEAR_BUILT,geometry
0,70_6_0,F_806090_2838651,FEE,70,ASSESS,NaN,20120104,NaN,N,83,...,450.227864,9797.818203,9320,"Vacant, Conservation (Municipal or County)",Public,CHURCH ST,E. BRIDGEWATER,TOWN OF EAST BRIDGEWATER,0,"POLYGON ((245656.133 865124.783, 245656.244 86..."
1,57_AB_07_22,F_551920_2858411,FEE,57A,ASSESS,NaN,20130325,NaN,N,226,...,155.367093,452.943514,930V,"IMPUTED - Vacant, Selectmen or City Council (M...",Public,SNEADE,OXFORD,TOWN OF OXFORD,0,"POLYGON ((168247.138 871215.679, 168242.175 87..."
2,56_A_06,F_555650_2856051,FEE,56,ASSESS,NaN,20130325,NaN,N,226,...,312.810050,4877.859755,930V,"IMPUTED - Vacant, Selectmen or City Council (M...",Public,MAIN,OXFORD,TOWN OF OXFORD,0,"POLYGON ((169424.723 870541.533, 169422.964 87..."
3,56_C_15,F_558185_2857434,FEE,56,ASSESS,NaN,20130325,NaN,N,226,...,963.411427,42233.845786,930V,"IMPUTED - Vacant, Selectmen or City Council (M...",Public,INDUSTRIAL PARK EAST,OXFORD,TOWN OF OXFORD,0,"POLYGON ((170279.891 871063.402, 170263.908 87..."
4,56_C_15_11,F_559045_2856647,FEE,56,ASSESS,NaN,20130325,NaN,N,226,...,289.938458,4576.467216,930V,"IMPUTED - Vacant, Selectmen or City Council (M...",Public,INDUSTRIAL PARK EAST,OXFORD,TOWN OF OXFORD,0,"POLYGON ((170447.351 870728.178, 170431.142 87..."


In [10]:
public_inst_excluded_land.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 88327 entries, 0 to 88326
Data columns (total 26 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   MAP_PAR_ID  85979 non-null  object  
 1   LOC_ID      88327 non-null  object  
 2   POLY_TYPE   88327 non-null  object  
 3   MAP_NO      85114 non-null  object  
 4   SOURCE      88327 non-null  object  
 5   PLAN_ID     5718 non-null   object  
 6   LAST_EDIT   88327 non-null  int64   
 7   BND_CHK     4727 non-null   object  
 8   NO_MATCH    88327 non-null  object  
 9   TOWN_ID     88327 non-null  int64   
 10  SITE        0 non-null      float64 
 11  ESN         88327 non-null  int64   
 12  LU_CODES    0 non-null      float64 
 13  DEV         0 non-null      float64 
 14  SYM1        88327 non-null  int64   
 15  SYM2        88327 non-null  int64   
 16  SHAPE_Leng  88327 non-null  float64 
 17  SHAPE_Area  88327 non-null  float64 
 18  USE_CODE    88327 non-null  object  
 

In [12]:
#filter to only include groups with "public" in name
#public_inst_excluded_land.GROUP_.unique()

public_groups = ['Public', 'Museum or library, Public', 'Education, Public', 'ROW potential',
                 'Public, Education', 'cemetery, Public', 'Public, Cemetery', 'Hospital, Public']

public_land = public_inst_excluded_land.loc[public_inst_excluded_land['GROUP_'].isin(public_groups)]


In [15]:
franklin_tess_gdb = 'K:\\DataServices\\Projects\\Current_Projects\\Environment\\MS4\\Project\\MS4_Model.gdb'
franklin_tess = gpd.read_file(franklin_tess_gdb, layer='franklin_tessellation')


In [26]:
town_name = "Franklin"
state_muni_parcels = get_landuse_data(town_name)
state_muni_parcels['muni'] = town_name
state_muni_parcels['type'] = 'parcel'

In [20]:
franklin_row_gdb = 'K:\DataServices\Projects\Current_Projects\Environment\MS4\Project\ROW_model_output.gdb'
franklin_row = gpd.read_file(franklin_row_gdb, layer ='ROW_segmentation_Franklin')

In [21]:
state_muni_parcels.head()

,parloc_id,Address,Owner,UseCodes,UseDesc,TRANSIT,ACRES,SQFT,PublicInst,NonPubExc,...,Description__short_,More_complete_description,ObjectID,Shape_Length,Shape_Area,non_exc,exc_acres,acreage,developable_land,pct_developable
0,M_209029_875915,"217 BENT ST, BENT ST, FRANKLIN, 02038",DAVIS JAMES W,109,Multiple Houses on one parcel,N,2.403956,104716.334888,0.0,0.0,...,Residential Other,Multiple houses on one parcel,15.0,447.583805,9728.474510,2.403956,0.0,2.403956,2.403956,1.0
1,M_208979_875894,"205 BENT ST, BENT ST, FRANKLIN, 02038",GOULET BRANDON,101,Single Family Residential,N,0.748742,32615.220544,0.0,0.0,...,Single family residence,Single family,4.0,240.582248,3030.045276,0.748742,0.0,0.748742,0.748742,1.0
2,M_208954_875864,"201 BENT ST, BENT ST, FRANKLIN, 02038",CZYRYCA PETER,101,Single Family Residential,N,0.412226,17956.554041,0.0,0.0,...,Single family residence,Single family,4.0,173.424348,1668.220899,0.412226,0.0,0.412226,0.412226,1.0
3,M_208993_875837,"9 MECHANIC PL, MECHANIC PL, FRANKLIN, 02038",CARMICHAEL RICHARD L,101,Single Family Residential,N,0.671510,29250.959052,0.0,0.0,...,Single family residence,Single family,4.0,221.646572,2717.506872,0.671510,0.0,0.671510,0.671510,1.0
4,M_209047_875824,"11 MECHANIC PL, MECHANIC PL, FRANKLIN, 02038",BAGLEY MICHAEL E,101,Single Family Residential,N,1.361988,59328.202176,0.0,0.0,...,Single family residence,Single family,4.0,365.903650,5511.765952,1.361988,0.0,1.361988,1.361988,1.0


In [25]:
franklin_row = franklin_row[['parloc_id', 'poly_typ', 'geometry']]
franklin_row['row_ID'] = 
franklin_row['type'] = 'ROW segment'
 

,parloc_id,poly_typ,geometry
0,M_209207_875817,PRIV_ROW,"MULTIPOLYGON (((209221.061 875795.082, 209223...."
1,M_208571_875882,PRIV_ROW,"MULTIPOLYGON (((208681.886 876034.959, 208681...."
2,M_208571_875882,PRIV_ROW,"MULTIPOLYGON (((208519.958 875991.594, 208518...."
3,M_208571_875882,PRIV_ROW,"MULTIPOLYGON (((208755.546 875779.894, 208754...."
4,M_208571_875882,PRIV_ROW,"MULTIPOLYGON (((208610.964 875775.762, 208610...."


In [ ]:
#merge together ROW parcels with parcel data from 3a
#then determine if they are public or not

#also grab tesselated data from gis, clip and do the same thing? how to do this?
